# Neural CSAD

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('ggplot')
%matplotlib inline

In [2]:
#Chargement des chunks en mémoires
size = 10000
df = pd.read_csv('./project_data/SET1V_01.CSV.gz', compression='gzip',names=['date','out_id', 'in_id', 'calls', 'duration'], parse_dates=['date'],sep=',', quotechar='"', chunksize=size)

In [3]:
# parcourir les chunks (attention à la ram)
for chunk in df:
    print(chunk)
    break

           date  out_id  in_id  calls  duration
0    2013-01-01       1      1      1        54
1    2013-01-01       1      2      1        39
2    2013-01-01       1     24      1      2957
3    2013-01-01       1    186      1        56
4    2013-01-01       2      2     22       418
5    2013-01-01       2      3      2        53
6    2013-01-01       2      4      4       455
7    2013-01-01       2      5      8       386
8    2013-01-01       2      6      7       204
9    2013-01-01       2      8     13       314
10   2013-01-01       2      9      2        60
11   2013-01-01       2     10      6       229
12   2013-01-01       2     11      2        27
13   2013-01-01       2     12      5       156
14   2013-01-01       2     13      3       101
15   2013-01-01       2     14      1        36
16   2013-01-01       2     15      2       454
17   2013-01-01       2     16      6       252
18   2013-01-01       2     19      3       140
19   2013-01-01       2     22      3   